In [1]:
import datetime
import fetcher
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2023, 1, 1)
#endDate = datetime.date(2024, 7, 2)
endDate = datetime.date.today()
Stocks_list = []

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS/US High ADR_2024-07-24.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)


focus_list_vol = {}
focus_list_ADR = {}
focus_list_3Month = {}
focus_list_1Month = {}
focus_list_RS_min_max = {}
focus_list_EMA10 = []
focus_candleSize = {}
focus_RS_52weekLow = []
focus_RS_52weekHigh = []

index_df = fetcher.__download_data( ["QQQ"], startDate, endDate, "US")

for ticker in Stocks_list:

    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    #Neglecting tickers with len < 66

    if len(df) > 200:
        if TA.SMA(df, period = 200)[-1] > df["Close"][-1]:
            continue

    if len(df) < 30:
        continue
    
    # Avg Volume*Price
    df_temp = df.iloc[-30:]
    avg_dollar_vol = 0
    for index, row in df_temp.iterrows():
        avg_dollar_vol += (
            df_temp.loc[index, "Volume"] * df_temp.loc[index, "Close"]
        ) / 30
    if avg_dollar_vol >= 5000000:
        focus_list_vol[ticker] = avg_dollar_vol
 
    # ADR%
    df_temp = df.iloc[-20:]
    sum = 0
    for index, row in df_temp.iterrows():
        sum += (df_temp.loc[index, "High"] / df_temp.loc[index, "Low"]) / 20

    ADR = 100 * (sum - 1)
    if ADR >= 3.75:
        focus_list_ADR[ticker] = ADR
   
    # Low near to or less than EMA10
    if df["Low"][-1] <= (TA.EMA(df, period = 10)[-1]): #
        focus_list_EMA10.append(ticker)

    #Check candle size less than ADR%
    stopLoss = -1*(df["Low"][-1]/df["High"][-1] -1)*100
    if stopLoss <= ADR*1.5 and stopLoss <= 7: #
        focus_candleSize[ticker] = stopLoss
    
    #Relative strength 52week lows in last 2 months
    if len(df) >= 260: #If the stock has less than 260 days, just append it
        index_df_temp = index_df.iloc[-260:]
        df_temp = df.iloc[-260:]
        RS = df_temp["Close"].values/index_df_temp["Close"].values
        Min = RS[0]
        min_index = 0
        for i in range(1, len(df_temp)):
            if RS[i]<Min:
                Min = RS[i]
                min_index = i
        
        if 260 - min_index >= 44:
            focus_RS_52weekLow.append(ticker)

    else:
        focus_RS_52weekLow.append(ticker)

    #Relative strength 52week highs in last 2 months
    if len(df) >= 260: #If the stock has less than 260 days, just append it
        index_df_temp = index_df.iloc[-260:]
        df_temp = df.iloc[-260:]
        RS = df_temp["Close"].values/index_df_temp["Close"].values
        Max = RS[0]
        max_index = 0
        for i in range(1, len(df_temp)):
            if RS[i]>Max:
                Max = RS[i]
                max_index = i
        
        if 260 - max_index <= 66:
            focus_RS_52weekHigh.append(ticker)

    else:
        focus_RS_52weekHigh.append(ticker)

    # 1Month% performance
    """df_temp = df.iloc[-22:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_1Month[ticker] = max_change_perc"""


    # 3Month% performance
    
    """df_temp = df.iloc[-66:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_3Month[ticker] = max_change_perc"""

    # Relative Strength low-high ratio 1M
    """index_df_temp = index_df.iloc[-22:]
    df_temp = df.iloc[-22:]
    RS = df_temp["Close"].values/index_df_temp["Close"].values
    min = RS[0]
    max = RS[1]
    for i in range(len(RS)-1):
        for j in range(i+1, len(RS)):
            if RS[j] > RS[i]:
                if (RS[j]-RS[i])/RS[i] * 100 > (max-min)/min *100:
                    max = RS[j]
                    min = RS[i]

    RS_change_perc = (max-min)/min * 100

    focus_list_RS_min_max[ticker] = RS_change_perc"""
    
final_focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_RS_52weekHigh) #& set(focus_candleSize) #& set(focus_list_EMA10)
print(final_focus)

reading from file:  yahoo_data/US/QQQ2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/VZIO2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/ZS2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/METC2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/RKLB2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/DSX2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/CAMT2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/HNRG2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/ARKO2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/AHCO2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/AVO2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/BVS2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/NVTS2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/ALTG2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/WOR2023-01-012024-07-24.csv
reading from file:  yahoo_data/US/APP2023-01-012024-07-24.csv


## Filtering

In [2]:
focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_RS_52weekLow)#& set(focus_candleSize.keys()) # 

ADR_dict = {}
avg_dollar_vol_dict = {}
#Perf_3M = {}
#RS_min_max = {}
#Perf_1M = {}


for ticker in focus:
    ADR_dict[ticker] = focus_list_ADR[ticker]
    avg_dollar_vol_dict[ticker] = focus_list_vol[ticker]
    #Perf_3M[ticker] = focus_list_3Month[ticker]
    #RS_min_max[ticker] = focus_list_RS_min_max[ticker]
    #Perf_1M[ticker] = focus_list_1Month[ticker]
    

In [3]:
len(focus)

189

## Grouping by Industries

In [4]:
industries = {}
for ticker in focus:
    if stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0] not in industries.keys():
        industries[stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]] = []
    industries[stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]].append(ticker)

industries = dict(sorted(industries.items(), key=lambda i: -len(i[1])))

## Ranking by dol_vol for each inds

In [5]:
for inds in industries:
    avg_dollar_vol_dict_tmp = {}
    for ticker in industries[inds]:
        avg_dollar_vol_dict_tmp[ticker] = avg_dollar_vol_dict[ticker]
    industries[inds] = [ticker for ticker, _ in sorted(avg_dollar_vol_dict_tmp.items(), key=lambda item: item[1], reverse=True)]

In [6]:
bocus = []
for inds in industries:
    bocus.extend(industries[inds])
print(bocus)

['LNTH', 'TMDX', 'TNDM', 'NARI', 'PRCT', 'STAA', 'NEOG', 'NVCR', 'ESTA', 'CDNA', 'NNOX', 'ZIMV', 'BLFS', 'CELC', 'NVDA', 'AMD', 'ARM', 'FSLR', 'RUN', 'ALAB', 'PI', 'SMTC', 'FORM', 'SGH', 'SITM', 'AOSL', 'HIMX', 'PLTR', 'APP', 'SNAP', 'SOUN', 'AI', 'ZETA', 'COMP', 'PHR', 'ETWO', 'BAND', 'SPIR', 'MARA', 'CLSK', 'IREN', 'CORZ', 'WULF', 'HUT', 'BITF', 'CIFR', 'APLD', 'GDS', 'INOD', 'GEV', 'ENVX', 'BE', 'SMR', 'POWL', 'AMSC', 'EOSE', 'MSTR', 'RDDT', 'JMIA', 'MMYT', 'EVER', 'GDRX', 'CRDO', 'RBRK', 'HIVE', 'PBI', 'BLND', 'SOI', 'FIX', 'STRL', 'IESC', 'AMRC', 'TPC', 'ORN', 'DESP', 'BLNK', 'XPOF', 'FIP', 'APEI', 'BTBT', 'GEO', 'RPAY', 'GOTU', 'HUYA', 'FTAI', 'RKLB', 'LOAR', 'LUNR', 'VITL', 'CGC', 'ACB', 'SNDL', 'TSLA', 'RIVN', 'BLBD', 'REVG', 'HIMS', 'GH', 'CLDX', 'WGS', 'CVNA', 'WSM', 'EVGO', 'HZO', 'CDE', 'AG', 'EXK', 'GATO', 'RKT', 'AHCO', 'TREE', 'COIN', 'HOOD', 'FUTU', 'ARLO', 'COMM', 'AIOT', 'CLS', 'CAMT', 'RXST', 'GGAL', 'CUBI', 'SFBS', 'CAVA', 'EAT', 'SG', 'SMCI', 'DELL', 'NATL', 'CEG',

In [7]:
industries

{'Medical specialties': ['LNTH',
  'TMDX',
  'TNDM',
  'NARI',
  'PRCT',
  'STAA',
  'NEOG',
  'NVCR',
  'ESTA',
  'CDNA',
  'NNOX',
  'ZIMV',
  'BLFS',
  'CELC'],
 'Semiconductors': ['NVDA',
  'AMD',
  'ARM',
  'FSLR',
  'RUN',
  'ALAB',
  'PI',
  'SMTC',
  'FORM',
  'SGH',
  'SITM',
  'AOSL',
  'HIMX'],
 'Packaged software': ['PLTR',
  'APP',
  'SNAP',
  'SOUN',
  'AI',
  'ZETA',
  'COMP',
  'PHR',
  'ETWO',
  'BAND',
  'SPIR'],
 'Data processing services': ['MARA',
  'CLSK',
  'IREN',
  'CORZ',
  'WULF',
  'HUT',
  'BITF',
  'CIFR',
  'APLD',
  'GDS',
  'INOD'],
 'Electrical products': ['GEV', 'ENVX', 'BE', 'SMR', 'POWL', 'AMSC', 'EOSE'],
 'Internet software/Services': ['MSTR',
  'RDDT',
  'JMIA',
  'MMYT',
  'EVER',
  'GDRX'],
 'Information technology services': ['CRDO',
  'RBRK',
  'HIVE',
  'PBI',
  'BLND',
  'SOI'],
 'Engineering & construction': ['FIX', 'STRL', 'IESC', 'AMRC', 'TPC', 'ORN'],
 'Other consumer services': ['DESP', 'BLNK', 'XPOF', 'FIP', 'APEI'],
 'Miscellaneous co

In [7]:
final_focus_ADR = {ticker: ADR for ticker, ADR in sorted(ADR_dict.items(), key=lambda item: item[1], reverse=True)}

In [8]:
final_focus_ADR.keys()

dict_keys(['GME', 'ASTS', 'SMR', 'NVAX', 'GPCR', 'CLSK', 'MARA', 'CIFR', 'HUT', 'ENVX', 'BE', 'SG', 'CGEM', 'ZIM', 'MSTR', 'POWL', 'SMCI', 'VKTX', 'COIN', 'SMTC', 'FSLR', 'GGAL', 'BMA', 'TDS', 'VST', 'GCT', 'MRNA', 'JANX', 'CAVA', 'GERN', 'TWST', 'VERA', 'MRUS', 'DYN', 'OSCR', 'NVCR', 'NXT', 'HOOD', 'GH', 'RXRX', 'CORT', 'DELL', 'CVNA', 'INDI', 'VRT', 'AI', 'BCRX', 'MOD', 'CDE', 'AGIO', 'ACMR', 'ENPH', 'CHWY', 'SOUN', 'ANF', 'ELF', 'VITL', 'DNLI', 'IESC', 'YPF', 'ASPN', 'LYFT', 'MMYT', 'APP', 'VIST', 'TMDX', 'IDYA', 'TNDM', 'SITM', 'WSM', 'CEG', 'RYTM', 'BILI', 'PTCT', 'JBLU', 'ACLX', 'CRSP', 'MPW', 'BROS', 'HBI', 'TGTX', 'CRNX', 'ASND', 'AG', 'STRL', 'CLS', 'SRPT', 'NRG', 'FTAI', 'MGNI', 'REVG', 'RXST', 'TOST', 'ESTC', 'NTNX', 'CAMT', 'SATS', 'AXSM', 'TDW', 'HL', 'CFLT', 'DUOL', 'FIX', 'DKNG', 'MRVL', 'LPG', 'IAG', 'ZETA', 'WIX', 'FSM', 'KRYS', 'AMD', 'ONON', 'ONTO', 'AMBA', 'BPMC', 'SKYW', 'RVMD', 'BLBD', 'NXE', 'PI', 'CNM', 'GNRC', 'VRNT', 'CCL', 'PSTG', 'FOUR', 'PAAS', 'PDD', 'RKT'

## Grouping by Sectors

In [37]:
sectors = {}
for ticker in focus:
    sector = stocks.loc[stocks['Symbol'] == ticker]["Sector"].values[0]
    if sector not in sectors.keys():
        sectors[sector] = []
    sectors[sector].append(ticker)

In [38]:
sectors

{'Retail Trade': ['ANF',
  'BABA',
  'SE',
  'DG',
  'BBWI',
  'PDD',
  'SIG',
  'TPR',
  'MELI',
  'MUSA',
  'URBN',
  'RH',
  'CVNA',
  'BJ',
  'DKS',
  'JD',
  'TGT',
  'AEO',
  'WSM',
  'VSCO',
  'CPNG',
  'JWN',
  'AAP',
  'W'],
 'Technology Services': ['RBA',
  'FICO',
  'DOCS',
  'DOCU',
  'PANW',
  'TOST',
  'TME',
  'CLSK',
  'GEN',
  'DOCN',
  'DUOL',
  'APP',
  'SNAP',
  'HUBS',
  'YMM',
  'BZ',
  'KD',
  'BILI',
  'TTD',
  'OKTA',
  'CYBR',
  'MNDY',
  'ESTC',
  'NFLX',
  'MSTR',
  'NTNX',
  'NU',
  'SOUN',
  'VRNS',
  'IOT',
  'APPF',
  'SQ',
  'WIX',
  'LYFT'],
 'Consumer Services': ['TKO',
  'PLNT',
  'FWONK',
  'TCOM',
  'PRKS',
  'EDU',
  'H',
  'NXST',
  'PLAY',
  'HRB',
  'CAVA',
  'FLUT',
  'CNK',
  'VVV'],
 'Producer Manufacturing': ['WMS',
  'AMAT',
  'HUBB',
  'MKSI',
  'ENVX',
  'TEX',
  'GTLS',
  'LII',
  'POWL',
  'AAON',
  'LRCX',
  'AYI',
  'ENTG',
  'ETN',
  'GE',
  'GTES',
  'ATMU',
  'MOD',
  'AZEK',
  'GNRC'],
 'Finance': ['MPW',
  'OWL',
  'BAP',
  'URI

In [32]:
sectors = {}
for ticker in Stocks_list:
    sector = stocks.loc[stocks['Symbol'] == ticker]["Sector"].values[0]
    industry = stocks.loc[stocks['Symbol'] == ticker]["Industry"].values[0]
    if sector not in sectors.keys():
        sectors[sector] = []
    if industry not in sectors[sector]:
        sectors[sector].append(industry)

In [33]:
sectors

{'Electronic Technology': ['Electronic Components',
  'Aerospace & Defense',
  'Semiconductors',
  'Telecommunications Equipment',
  'Computer Peripherals',
  'Computer Processing Hardware',
  'Electronic Production Equipment',
  'Electronic Equipment/Instruments'],
 'Commercial Services': ['Miscellaneous Commercial Services',
  'Commercial Printing/Forms',
  'Advertising/Marketing Services',
  'Personnel Services',
  'Financial Publishing/Services'],
 'Technology Services': ['Information Technology Services',
  'Packaged Software',
  'Internet Software/Services',
  'Data Processing Services'],
 'Producer Manufacturing': ['Industrial Machinery',
  'Electrical Products',
  'Building Products',
  'Miscellaneous Manufacturing',
  'Office Equipment/Supplies',
  'Industrial Conglomerates',
  'Trucks/Construction/Farm Machinery',
  'Auto Parts: OEM',
  'Metal Fabrication'],
 'Health Technology': ['Biotechnology',
  'Pharmaceuticals: Major',
  'Pharmaceuticals: Other',
  'Medical Specialties'

## Ranking by ADR 

In [4]:
final_focus_ADR = {ticker: ADR for ticker, ADR in sorted(ADR_dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
final_focus_ADR.keys()

In [ ]:
for ticker in final_focus_ADR.keys():
    print(ticker, final_focus_ADR[ticker], Perf_1M[ticker])

## Ranking by Performance 3M

In [56]:
final_focus_3M = {ticker: perf for ticker, perf in sorted(Perf_3M.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
final_focus_3M.keys()

In [ ]:
for ticker in final_focus_3M.keys():
    print(ticker, final_focus_3M[ticker], ADR_dict[ticker], focus_candleSize[ticker])

## Ranking by Performance 1M

In [68]:
final_focus_1M = {ticker: perf for ticker, perf in sorted(Perf_1M.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
final_focus_1M.keys()

In [ ]:
for ticker in final_focus_1M.keys():
    print(ticker, final_focus_1M[ticker], ADR_dict[ticker], focus_candleSize[ticker])